# Adding satellite accounts

This tutorial shows how to possibly add satellite accounts to any mario.Database object

## Background

Input-output tables are not necessarily enriched with satellite accounts (i.e. environmentally-extended). However, in the field of industrial ecology and life-cycle assessment (LCA), it is paramount each industrial activity is characterized also in terms of its interactions with the external environment. Satellite accounts may cover also non-environmental information, such as employment (e.g. employed people by each activity)

Having the possibility to extend IOTs and SUTs to additional accounts may be helpful for many analysts and modellers.

## Adding extensions on a custom IOT

Starting from the test IOT matrix of mario, we can see the table has four satellite accounts

In [ ]:
import mario  # Import MARIO

test_IOT = mario.load_test(table='IOT')
test_IOT.get_index('Satellite account')

### Preparing data in Excel

Let's assume we desire to extend the table to account for "CH4" emissions. First, we need to prepare the data in the proper format in Excel. MARIO provides an Excel template by calling the 'get_extensions_excel' method

In [ ]:
path = 'add_extensions/new_E_accounts.xlsx'
test_IOT.get_extensions_excel(matrix='E',path=path)

The data is then filled with the desired information, in this case regarding CH4 emissions. These data needs to be collected separately by the analyst.

![Alt text](add_extensions\new_E.jpg)

This data can be parsed using pandas 'read_excel' method. In this case the data are stored in the file targeted by the given path and in the sheet named "E flows". 

It is also important to provide the unit of measure of the new accounts, that must be provided as reported also in the next Figure.

![Alt text](add_extensions\units.jpg)

The units can be parsed using pandas, by adding a 'units' sheet to the Excel file provided by MARIO or in any other file

In [ ]:
import pandas as pd

units = pd.read_excel(path, sheet_name='units',index_col=[0],header=[0])
units

## Adding the new satellite accounts
Once both data and units are parsed, just call the "add_extensions" method of the mario.Database class and provide the required information

In [ ]:
test_IOT.add_extensions(
    io=path,
    units=units,
    matrix='E'
    )

## Adding extensions to Exiobase monetary SUTs from corresponding IOTs

Another relevant application of the 'add_extensions' method is related to adding extensions to the Monetary SUT version of Exiobase, which are not environmentally-extended when downloaded from Zenodo. 
However, they have the same granularity as their corresponding activity-by-activity IOT in terms of regions and sectors (activities in case of the SUT). 

By downloading both SUT and IOT tables, it is possible to get the satellite accounts from the latter to the former.

### Parse both Exiobase tables
First, start by parsing both tables (referred to 2019 in this example): refer to the EXIOBASE parser tutorial for further information

In [1]:
import mario

sut_path = 'EXIOBASE_folder\MRSUT_2019.zip'
iot_path = 'EXIOBASE_folder\IOT_2019_ixi.zip'

sut = mario.parse_exiobase(table='SUT',unit='Monetary',path=sut_path)
iot = mario.parse_exiobase(table='IOT',unit='Monetary',path=iot_path)

<>:3: SyntaxWarning: invalid escape sequence '\M'
<>:4: SyntaxWarning: invalid escape sequence '\I'
<>:3: SyntaxWarning: invalid escape sequence '\M'
<>:4: SyntaxWarning: invalid escape sequence '\I'
C:\Users\loren\AppData\Local\Temp\ipykernel_3556\1850789910.py:3: SyntaxWarning: invalid escape sequence '\M'
  sut_path = 'EXIOBASE_folder\MRSUT_2019.zip'
C:\Users\loren\AppData\Local\Temp\ipykernel_3556\1850789910.py:4: SyntaxWarning: invalid escape sequence '\I'
  iot_path = 'EXIOBASE_folder\IOT_2019_ixi.zip'
cvxpy module is not installed in your system. This will raise problems in some of the abilities of MARIO


The next step would be to append to create a new E matrix for the SUT starting from the E matrix of the IOT. 
Consider that the indices would be different since the IOT would have a "Sector" level instead of "Activity" and "Commodity" level as for the SUT. 

The new E of the SUT will be initialized as an empty dataframe, filled with zeros, having the same indices as the IOT's E and the same columns as the SUT's E.

In [3]:
import pandas as pd

E_iot = iot.E
E_sut = sut.E

new_E_sut = pd.DataFrame(0, index=E_iot.index, columns=E_sut.columns)

It is then necessary to fill the values of the IOT's E into the new SUT's E. Before this, it is safer to change the second level of the columns index level of the IOT's E into 'Activity'

In [5]:
new_column_levels = pd.MultiIndex.from_arrays([
    E_iot.columns.get_level_values(0),
    ['Activity' for i in range(E_iot.shape[1])],
    E_iot.columns.get_level_values(2)
])

E_iot.columns = new_column_levels

At this point, it is enough to update the new SUT's E with values of the IOT's E

In [6]:
new_E_sut.update(E_iot)

Regarding the units, they will be taken from the IOT as well

In [7]:
new_units_sut = iot.units['Satellite account']

To conclude, just call the 'add_extensions' method as shown for the simplified case

In [8]:
sut.add_extensions(
    io=new_E_sut,
    units=new_units_sut,
    matrix='E'
)

Using add extensions will rewrite the new results on the baseline and delete other scenarios
C:\Users\loren\Documents\GitHub\MARIO-Org\MARIO\mario\core\AttrData.py:410: SyntaxWarning:

invalid escape sequence '\{'

C:\Users\loren\Documents\GitHub\MARIO-Org\MARIO\mario\core\AttrData.py:1029: SyntaxWarning:

invalid escape sequence '\s'



WrongInput: units dataframe should has exactly the same index levels of io